In [1]:
import logging
#Logging is a means of tracking events that happen when some software runs
# Create and configure logger
logging.basicConfig(filename="../logs/newfile.log",
                    format='%(asctime)s %(message)s',
                    filemode='w')
 
# Creating an object
logger = logging.getLogger()
 
# Setting the threshold of logger to DEBUG
logger.setLevel(logging.DEBUG)
 
# Test messages
logger.debug("Harmless debug Message")
logger.info("Just an information")
logger.warning("Its a Warning")
logger.error("Did you try to divide by zero")
logger.critical("Internet is down")

In [4]:
import sys
sys.path.insert(0,'../scripts/')
from data_loader import load_df_from_csv
from data_struct import DataInfo
from data_cleaner import DataCleaner
%matplotlib inline


In [5]:
missing_values = ["n/a", "na", "undefined",'']
df= load_df_from_csv("../data/causal_data.csv",na_values=missing_values)

In [6]:
explorer = DataInfo(df)
#initialize module to get data overview

In [7]:
explorer.get_basic_description()

The DataFrame containes 569 rows and 9 columns.
Current DataFrame Memory Usage:
17767
Current DataFrame Memory Usage of columns is :
DataFrame Information: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            569 non-null    uint16 
 1   radius_mean           569 non-null    float32
 2   area_mean             569 non-null    float32
 3   concave points_mean   569 non-null    float32
 4   radius_worst          569 non-null    float32
 5   perimeter_worst       569 non-null    float32
 6   area_worst            569 non-null    float32
 7   concave points_worst  569 non-null    float32
 8   diagnosis_Cat         569 non-null    uint8  
dtypes: float32(7), uint16(1), uint8(1)
memory usage: 17.4 KB


In [8]:
cleaner = DataCleaner(df)

In [9]:
cleaner.remove_unwanted_columns("Unnamed: 0")

,radius_mean,area_mean,concave points_mean,radius_worst,perimeter_worst,area_worst,concave points_worst,diagnosis_Cat
0,17.990000,1001.000000,0.14710,25.379999,184.600006,2019.000000,0.2654,1
1,20.570000,1326.000000,0.07017,24.990000,158.800003,1956.000000,0.1860,1
2,19.690001,1203.000000,0.12790,23.570000,152.500000,1709.000000,0.2430,1
3,11.420000,386.100006,0.10520,14.910000,98.870003,567.700012,0.2575,1
4,20.290001,1297.000000,0.10430,22.540001,152.199997,1575.000000,0.1625,1
...,...,...,...,...,...,...,...,...
564,21.559999,1479.000000,0.13890,25.450001,166.100006,2027.000000,0.2216,1
565,20.129999,1261.000000,0.09791,23.690001,155.000000,1731.000000,0.1628,1
566,16.600000,858.099976,0.05302,18.980000,126.699997,1124.000000,0.1418,1
567,20.600000,1265.000000,0.15200,25.740000,184.600006,1821.000000,0.2650,1


In [10]:
explorer.get_basic_description()

The DataFrame containes 569 rows and 8 columns.
Current DataFrame Memory Usage:
16629
Current DataFrame Memory Usage of columns is :
DataFrame Information: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   radius_mean           569 non-null    float32
 1   area_mean             569 non-null    float32
 2   concave points_mean   569 non-null    float32
 3   radius_worst          569 non-null    float32
 4   perimeter_worst       569 non-null    float32
 5   area_worst            569 non-null    float32
 6   concave points_worst  569 non-null    float32
 7   diagnosis_Cat         569 non-null    uint8  
dtypes: float32(7), uint8(1)
memory usage: 16.2 KB


In [12]:
explorer.get_basic_description()

The DataFrame containes 569 rows and 8 columns.
Current DataFrame Memory Usage:
16629
Current DataFrame Memory Usage of columns is :
DataFrame Information: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   radius_mean           569 non-null    float32
 1   area_mean             569 non-null    float32
 2   concave points_mean   569 non-null    float32
 3   radius_worst          569 non-null    float32
 4   perimeter_worst       569 non-null    float32
 5   area_worst            569 non-null    float32
 6   concave points_worst  569 non-null    float32
 7   diagnosis_Cat         569 non-null    uint8  
dtypes: float32(7), uint8(1)
memory usage: 16.2 KB
